# Evalutation of the 200_sashasbender Sizescan

extract the emittances for all offsets and safe in a csv

In [14]:
import os
import numpy as np
import pandas as pd
import altair as alt
alt.enable_mime_rendering()
import scipy.constants

import PhaseSpaceEval.monitor_quantities as monq
from PhaseSpaceEval.import_particle_data import *
from PhaseSpaceEval.trajectory import Trajectory
from PhaseSpaceEval.particlemonitor import ParticleMonitor

## Import Raw Data and set filename *!!! ADJUST FILENAMES HERE !!!*

In [15]:
MODELNAME = "200_sashasbender_sizescan"
RAW_PATH = "rawdata_" + MODELNAME + "/"
EMIT_FILENAME = "sc_emit_" + MODELNAME + ".csv" # Name for the emittance output file
#MON_FILENAME = "mon_" + MODELNAME + ".csv" # Name for the monitors output file

particle_source_names = import_source_names(RAW_PATH + MODELNAME + "-source_names.txt")
particle_constants = import_particle_constants(RAW_PATH + MODELNAME + "-constants.txt")
particle_trajectories = import_particle_trajectories(RAW_PATH + MODELNAME + "-trajectories.txt")

## Create convenient lists
### Raw Data

In [16]:
# Delete the single_centre source, not required
for key in particle_source_names.keys():
    if particle_source_names[key] == "single_centre":
        del(particle_source_names[key])
        break
        
# Generate simple list with all source IDs
sourceIDs = list(particle_source_names.keys())
#print(sourceIDs)

# Create lists with all particles belonging to a source and with the id of the central particles
particlesBySrc = dict() # dict for all particleIDs
centresBySrc = dict() # dict with the ids of the central particles
for sID in sourceIDs:
    pIDs = particle_constants["particleID"].loc[particle_constants["sourceID"] == sID].tolist()
    particlesBySrc.update({sID : pIDs})
    centresBySrc.update({sID : min(pIDs)}) # the smallest pID for each source is the centre
#print(particlesBySrc)
#print(centresBySrc)

### Trajectories

In [17]:
# Create Trajectories
trajsBySrc = dict() # Dict for all trajectories
ctrajsBySrc = dict() # Dict for central trajectories
lostParticles = list()
for sID in sourceIDs:
    # Compute central trajectory for each sID
    cID = centresBySrc[sID]
    ctr = Trajectory(particle_trajectories[cID],
                     particle_constants.loc[particle_constants["particleID"] == cID].squeeze())
    ctrajsBySrc.update({sID : ctr})

    # For each sID compute the trajectories of all pIDs
    # Note Particles that cannot be found in trajectory dataframe, these were lost
    pIDs = particlesBySrc[sID]
    trajs = list()
    for pID in pIDs:
        try:
            tr = Trajectory(particle_trajectories[pID],
                            particle_constants.loc[particle_constants["particleID"] == pID].squeeze())
        except KeyError:
            lostParticles.append(pID)
        trajs.append(tr)
    trajsBySrc.update({sID : trajs})
print(lostParticles)

[2750, 2784, 3028, 3079, 3094, 3160, 3195, 3341, 3697, 3717, 3790, 3794, 3799, 3812, 3923, 4028, 4067, 4119, 4147, 4150, 4155, 4165, 4326, 4331, 4349, 4370, 4409, 4422, 4446, 4453, 4467, 4487, 4499, 4554, 4566, 4606, 4690, 4701, 4734, 4736, 4765, 4814, 4848, 4851, 4859, 4877, 4897, 4908, 4933, 5007, 5018, 5042, 5067, 5078, 5080, 5113, 5120, 5183, 5211, 5241, 5248, 5254, 5264, 5270, 5274, 5286, 5303, 5315, 5324, 5331, 5366, 5379, 5393, 5397, 5422, 5457, 5499, 5504, 5576, 5584, 5592, 5624, 5629, 5662, 5670, 5683, 5686, 5710, 5720, 5736, 5741, 5746, 5766, 5787, 5789, 5800, 5803, 5810, 5837, 5841, 5848, 5850, 5856, 5861, 5889, 5890, 5896, 5898, 5900, 5901, 5914, 5926, 5928, 5934, 5943, 5944, 5962, 6004, 6014, 6021, 6030, 6033, 6040, 6052, 6060, 6062, 6066, 6071, 6073, 6080, 6084, 6088, 6089, 6094, 6097, 6103, 6105, 6106, 6111, 6112, 6115, 6134, 6136, 6140, 6142, 6143, 6154, 6157, 6161, 6163, 6167, 6177, 6181, 6186, 6189, 6191, 6192, 6194, 6197, 6199, 6215, 6234, 6236, 6239, 6244, 6245, 625

### Monitors

In [18]:
# Create Monitors
monBySrc = dict()
for sID in sourceIDs:
    ctr = ctrajsBySrc[sID]
    t0 = ctr.find_time("z", 501)
    mon = ParticleMonitor(time0=t0, trajectory=ctr)
    monBySrc.update({sID : mon})

## Record Monitor Interactions

In [19]:
# Record Monitor Interactions
for sID in sourceIDs:
    mon = monBySrc[sID]
    mon.reset_events()
    mon.reset_misses()
    for tr in trajsBySrc[sID]:
        mon.record_intersect(tr)

# Read Out misses and events
missesBySrc = dict()
eventsBySrc = dict()
for sID in sourceIDs:
    missesBySrc.update({sID : monBySrc[sID].get_misses()})
    eventsBySrc.update({sID : monBySrc[sID].get_events()})
#print(missesBySrc)

# Add lost particles to miss counter
for pID in lostParticles:
    for sID in sourceIDs:
        if pID in particlesBySrc[sID]:
            missesBySrc[sID] += 1
#print(missesBySrc)

## Export Monitor Interactions

In [20]:
'''export_events = pd.DataFrame()
for sID in sourceIDs:
    events = eventsBySrc[sID].copy()
    events["sourceID"] = sID
    events["sourceName"] = particle_source_names[sID]
    export_events = export_events.append(events, ignore_index=True)
export_events.to_csv(MON_FILENAME)'''

'export_events = pd.DataFrame()\nfor sID in sourceIDs:\n    events = eventsBySrc[sID].copy()\n    events["sourceID"] = sID\n    events["sourceName"] = particle_source_names[sID]\n    export_events = export_events.append(events, ignore_index=True)\nexport_events.to_csv(MON_FILENAME)'

## Compute and save emittances

In [21]:
# Compute and save emittances
colnames = ["sourceID", "sourceName", "beamsize", "x_emittance", "y_emittance",
            "x_norm_emittance", "y_norm_emittance", "losses", "relbeta"]
emit_df = pd.DataFrame(columns=colnames)
emit_temp = pd.DataFrame([np.zeros(len(colnames))], columns=colnames)
for sID in sourceIDs:
    name = particle_source_names[sID]
    bs = float(name.split('_')[1])
    xemit = monq.emittance_u(eventsBySrc[sID])
    yemit = monq.emittance_v(eventsBySrc[sID])
    # Compute rel. beta for the screen 1e6 mm/ns ->m/s
    beta = monBySrc[sID].abs_vel / scipy.constants.speed_of_light * 1e6
    xemit_n = beta * (1 - beta**2)**(-0.5) * xemit # beta * gamme * emit
    yemit_n = beta * (1 - beta**2)**(-0.5) * yemit
    losses = missesBySrc[sID]
    
    emit_temp["sourceID"] = sID
    emit_temp["sourceName"] = name
    emit_temp["beamsize"] = bs
    emit_temp["x_emittance"] = xemit
    emit_temp["y_emittance"] = yemit
    emit_temp["x_norm_emittance"] = xemit_n
    emit_temp["y_norm_emittance"] = yemit_n
    emit_temp["losses"] = losses
    emit_temp["relbeta"] = beta

    emit_df = emit_df.append(emit_temp, ignore_index=True)

emit_df.sort_values(["beamsize"], inplace=True)
emit_df.reset_index(inplace=True, drop=True)

# Load emittance data from startinplane to compute relative emittance growth
#emit_start = pd.read_csv("emit_startingplane.csv", index_col=0)
#emit_df["x_emittance_rel"] = emit_df["x_emittance"]/emit_start["x_emittance"]
#emit_df["y_emittance_rel"] = emit_df["y_emittance"]/emit_start["y_emittance"]
# Save emittance table
emit_df.to_csv(EMIT_FILENAME)
# Show head for debugging purposes
emit_df

,sourceID,sourceName,beamsize,x_emittance,y_emittance,x_norm_emittance,y_norm_emittance,losses,relbeta
0,16777217,bs_1,1.0,0.003294,0.000053,0.000008,1.214114e-07,0,0.002309
1,16777218,bs_4,4.0,0.108426,0.007248,0.000250,1.673256e-05,0,0.002309
2,16777219,bs_7,7.0,0.660782,0.056931,0.001525,1.314267e-04,0,0.002309
3,16777220,bs_10,10.0,2.092301,1.004963,0.004830,2.319987e-03,0,0.002309
4,16777221,bs_13,13.0,6.010637,2.339256,0.013876,5.400241e-03,2,0.002309
5,16777222,bs_16,16.0,21.336979,15.215611,0.049257,3.512569e-02,13,0.002309
6,16777223,bs_19,19.0,44.994375,45.815338,0.103871,1.057661e-01,34,0.002309
7,16777224,bs_22,22.0,78.560890,86.327082,0.181360,1.992887e-01,69,0.002309
8,16777225,bs_25,25.0,89.032840,116.939793,0.205535,2.699589e-01,126,0.002309
9,16777226,bs_28,28.0,107.509548,128.035393,0.248189,2.955735e-01,167,0.002309


## Plots

In [22]:
alt.Chart(emit_df).mark_line().encode(
    x='beamsize',
    y='x_emittance',
)

<altair.VegaLite object>

In [23]:
alt.Chart(emit_df).mark_line().encode(
    x='beamsize',
    y='y_emittance',
)

<altair.VegaLite object>

In [24]:
alt.Chart(emit_df).mark_line().encode(
    x='beamsize',
    y='x_norm_emittance',
)

<altair.VegaLite object>

In [25]:
alt.Chart(emit_df).mark_line().encode(
    x='beamsize',
    y='y_norm_emittance',
)

<altair.VegaLite object>

In [26]:
alt.Chart(emit_df).mark_line().encode(
    x='beamsize',
    y='losses',
)

<altair.VegaLite object>